# Ensemble Learning: La sabiduría de la multitud.

## Introducción

Seguramente ha visto el siguiente experimento en algún documental de televisión: En un frasco grande se han introducido muchos objetos pequeños, por ejemplo caramelos, de tal manera que sea imposible contarlos "a ojo", luego se pide a muchas personas que estimen visualmente cuántos caramelos hay en el frasco. Las estimaciones de las personas suelen ser muy disímiles algunas estiman sólo centenas de caramelos y otras decenas de miles de caramelos; cada una de las estimaciones suele estar bastante alejada de la cantidad real de caramelos que hay en el frasco. Hasta aquí nada sorprendente, pero en un momento el presentador sugiere **promediar** las estimaciones dadas por todos los participantes y ... sorpresa, dicho promedio suele ser muy cercano a la cantidad real de caramelos que hay en el frasco!   

Lo invito a que vea el siguiente video https://youtu.be/od9MGUNjBVw al respecto.

Este experimento no es un truco televisivo, es real y se denomina "Sabiduría de la Multitud". Aunque las estimaciones individuales son bastante malas, al **ensamblar** todos los pronósticos individuales, por ejemplo calculando el promedio se suele obtener un resultado que es mejor que cualquier pronóstico individual!  Los errores que cometen los que subestiman la cantidad real se compensan con los errores de quienes sobreestiman dicha cantidad. 

> La clave en el experimento anterior es que **muchas** personas, lo **más distintas** posibles den su pronóstico.

El experimento puede adaptarse perfectamente a nuestros modelos de aprendizaje automático, la idea básica es entrenar varios modelos y luego, si es un problema de Regresión promediar sus pronósticos y si es un problema de Clasificación establecer un sistema de votación entre ellos. Los resultados del ensamblado suelen superar a los de cualquiera de los estimadores individuales.

Los ensamblados suelen ser los modelos que ganan en las competencias tipo Kaggle o el famosísimo Netflix Prize Competition y otras. Como desventaja podemos mencionar que, obviamente, consumen muchos más recursos computacionales que los métodos individuales.



## Ensemble para Problemas de Clasificación

Como comentábamos, para problemas de clasificación la idea es entrenar varios algoritmos, cuanto más sean en cantidad y diversidad, mejor y luego establecere agún tipo de votación.  

Se suelen utilizar dos mecanismos de votación, **hard voting** y **soft voting**.  

- **Hard Voting**: simplemente se asigna la clase que pronosticó la mayoría de los estimadores.   

- **Soft Voting**: en este caso es necesario que los estimadores utilizados pueden pronosticar **probabilidades**, como por ejemplo Regresión Logística y sus derivados, hablando en la jerga de Scikit - Learn, que nos ofrezcan predict_proba como resultado. En este caso se promedian las probabilidades pronosticadas para cada clase y se elige la clase que obtuvo el mayor valor promedio entre todos los modelos.   

> **Soft Voting suele dar mejores resultados que Hard Voting**


Afortunadamente todo esto se hace muy fácilmente con Scikit-Learn!

### VotingClassifier: Implementación en Scikit-Learn

Para instrumentar el mecanismo de votación Sci-kit Learn nos brinda

~~~~
sklearn.ensemble.VotingClassifier
~~~~ 

La documentación oficial la puede ver aquí: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

La sintaxis es la siguiente:  

~~~
sklearn.ensemble.VotingClassifier(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)
~~~

Los parámetros principales son: 

- **estimators**: lista de tuplas con el nombre de los modelos o estimadores previamente definidos, por ejemplo [('Regresión_Logística', rlog), ('k_Vecinos',knn) ...]  
- **voting**: hard o soft. Por defecto 'hard'. En el caso de empate sklearn elige el pronóstico del primer algoritmo en orden alfabético.
- **weights**: permite asignar un peso distinto a cada estimador 
- **n_jobs=None** Cantidad de hilos dedicados a esta tarea. **En este caso pueden obtenerse muy buenos resultados al paralelizar ya que puede correr cada uno de los modelos en un hilo distinto, se sugiere setear en -1**.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#### Los Datos

In [ ]:
df=pd.read_csv('data/winequality-white.csv', sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
X=df.drop(axis=1,columns='quality')
y=df['quality']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)

#### Los Modelos

Elijamos modelos que sean lo más distintos posible, por ejemplo:

- Regresión Logística con Ridge
- kNN
- Árbol
- SVM

##### Regresión Logística + Ridge

In [ ]:
ridge=linear_model.LogisticRegression(max_iter= 20000,penalty='l2',fit_intercept=True, random_state=123)

##### kNN

In [ ]:
knn=KNeighborsClassifier()

##### Árbol

In [ ]:
tree=DecisionTreeClassifier(random_state=123)

##### SVM

In [ ]:
svm=SVC( random_state=123)

##### Resultados individuales de Accuracy (la métrica por defecto para los clasificadores en sklearn)

In [ ]:
modelos_lista=(ridge, knn, tree, svm)

for modelo in modelos_lista:
    modelo.fit(X_train,y_train)
    AC=modelo.score(X_test,y_test)
    print("AC: ",AC)

AC:  0.560204081632653
AC:  0.49081632653061225
AC:  0.6020408163265306
AC:  0.4826530612244898


#### VotingClassifier: La votación

In [ ]:
from sklearn.ensemble import VotingClassifier

##### Creamos el VotingClassifier:

In [ ]:
modelos=[('ridge',ridge),('knn',knn),('arbol',tree),('svm',svm)]
votacion=VotingClassifier(estimators=modelos,voting='hard',n_jobs=-1, weights=[2,1,3,1])

##### Lo entrenamos con fit (como siempre ... gracias sklearn!)

In [ ]:
votacion.fit(X_train, y_train);

##### Lo evaluamos

In [ ]:
AC_votacion=votacion.score(X_test,y_test)
print("AC_votacion: ",AC_votacion)

AC_votacion:  0.6336734693877552


### Conclusión: 

En este caso el resultado del Ensemble mejoró ligeramente los resultados individuales.